In [14]:
import pandas as pd
csv_file_path = "dummy_npi_data.xlsx - Dataset.csv"
df = pd.read_csv(csv_file_path)


df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   NPI                       1000 non-null   int64 
 1   State                     1000 non-null   object
 2   Login Time                1000 non-null   object
 3   Logout Time               1000 non-null   object
 4   Usage Time (mins)         1000 non-null   int64 
 5   Region                    1000 non-null   object
 6   Speciality                1000 non-null   object
 7   Count of Survey Attempts  1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


(None,
           NPI State           Login Time          Logout Time  \
 0  1000000000    NY  2025-03-08 06:09:00  2025-03-08 06:28:00   
 1  1000000001    MI  2025-03-08 12:28:00  2025-03-08 13:10:00   
 2  1000000002    CA  2025-03-08 15:11:00  2025-03-08 15:37:00   
 3  1000000003    TX  2025-03-08 14:17:00  2025-03-08 15:36:00   
 4  1000000004    GA  2025-03-08 15:59:00  2025-03-08 17:37:00   
 
    Usage Time (mins)     Region   Speciality  Count of Survey Attempts  
 0                 19  Northeast   Cardiology                         3  
 1                 42    Midwest     Oncology                         5  
 2                 26       West     Oncology                         8  
 3                 79  Northeast  Orthopedics                         9  
 4                 98       West     Oncology                         0  )

In [15]:
df['Login Time'] = pd.to_datetime(df['Login Time'], errors='coerce')
df['Login Hour'] = df['Login Time'].dt.hour

In [16]:

df['Logout Time'] = pd.to_datetime(df['Logout Time'], errors='coerce')
df['Logout Hour'] = df['Logout Time'].dt.hour
df_corr = df.drop(columns=['Login Time', 'Logout Time'])
df_corr.corr(numeric_only=True)['Count of Survey Attempts'].sort_values(ascending=False)


Count of Survey Attempts    1.000000
Usage Time (mins)           0.008235
Login Hour                 -0.009265
Logout Hour                -0.012885
NPI                        -0.031769
Name: Count of Survey Attempts, dtype: float64

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Encode categorical features
label_encoders = {}
for col in ['State', 'Region', 'Speciality']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define features and target
X = df[['Login Hour', 'State', 'Region', 'Speciality', 'Usage Time (mins)']]
y = (df['Count of Survey Attempts'] > 0).astype(int)  # Convert to binary (0: No, 1: Yes)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.89

In [18]:
import joblib

# Save the trained model to a file
model_file_path = "random_forest_model.pkl"
joblib.dump(model, model_file_path)

['random_forest_model.pkl']

In [19]:

label_encoders_file_path = 'label_encoders.pkl'
joblib.dump(label_encoders, label_encoders_file_path)

['label_encoders.pkl']

In [20]:
df.head()

,NPI,State,Login Time,Logout Time,Usage Time (mins),Region,Speciality,Count of Survey Attempts,Login Hour,Logout Hour
0,1000000000,6,2025-03-08 06:09:00,2025-03-08 06:28:00,19,1,0,3,6,6
1,1000000001,4,2025-03-08 12:28:00,2025-03-08 13:10:00,42,0,3,5,12,13
2,1000000002,0,2025-03-08 15:11:00,2025-03-08 15:37:00,26,3,3,8,15,15
3,1000000003,9,2025-03-08 14:17:00,2025-03-08 15:36:00,79,1,4,9,14,15
4,1000000004,2,2025-03-08 15:59:00,2025-03-08 17:37:00,98,3,3,0,15,17
